In [1]:
import strawberryfields as sf
from strawberryfields.ops import S2gate, Dgate, MeasureX, MeasureP
import numpy as np

def calculate_covariance(X, Y):
    return np.mean(X * Y) - np.mean(X)*np.mean(Y)

def calculate_correlation(X, Y):
    cov = calculate_covariance(X, Y)
    std_X = np.sqrt(np.mean(X**2) - np.mean(X)**2)
    std_Y = np.sqrt(np.mean(Y**2) - np.mean(Y)**2)
    return cov / (std_X * std_Y) if (std_X>0 and std_Y>0) else np.nan

num_runs = 100000
r = 10.0

a = np.random.randint(0, 2, size=num_runs)
b = np.random.randint(0, 2, size=num_runs)

M_meas = []
M_prime_meas = []

for i in range(num_runs):
    eng = sf.Engine("gaussian")
    prog = sf.Program(2)

    with prog.context as q:
        S2gate(r) | (q[0], q[1])
        (MeasureX if a[i] == 0 else MeasureP) | q[0]
        (MeasureX if b[i] == 0 else MeasureP) | q[1]

    result = eng.run(prog)
    M_val = result.samples[0][0]
    M_prime_val = result.samples[0][1]

    M_meas.append(M_val)
    M_prime_meas.append(M_prime_val)

M_meas = np.array(M_meas)
M_prime_meas = np.array(M_prime_meas)

idx_00 = np.where((a == 0) & (b == 0))[0]
idx_11 = np.where((a == 1) & (b == 1))[0]
idx_01 = np.where((a == 0) & (b == 1))[0]
idx_10 = np.where((a == 1) & (b == 0))[0]


corr_00 = calculate_correlation(np.abs(M_meas[idx_00]), np.abs(M_prime_meas[idx_00]))
corr_11 = calculate_correlation(np.abs(M_meas[idx_11]), np.abs(M_prime_meas[idx_11]))
corr_01 = calculate_correlation(np.abs(M_meas[idx_01]), np.abs(M_prime_meas[idx_01]))
corr_10 = calculate_correlation(np.abs(M_meas[idx_10]), np.abs(M_prime_meas[idx_10]))

print(f"(a=0, b=0) => measure X, then displace+measure X: {corr_00:.4f}")
print(f"(a=1, b=1) => measure P, then displace+measure P: {corr_11:.4f}")
print(f"(a=0, b=1) => measure X, then displace+measure P: {corr_01:.4f}")
print(f"(a=1, b=0) => measure P, then displace+measure X: {corr_10:.4f}")


(a=0, b=0) => measure X, then displace+measure X: 1.0000
(a=1, b=1) => measure P, then displace+measure P: 1.0000
(a=0, b=1) => measure X, then displace+measure P: 0.0055
(a=1, b=0) => measure P, then displace+measure X: -0.0074
